In [1]:
import ROOT

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x55948a23a240
OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x55948a524bb0


In [2]:
%jsroot on

In [3]:
ROOT.EnableImplicitMT(4)
ROOT.TH1.SetDefaultSumw2()

In [4]:
ROOT.ildStyle.SetOptStat(0)
ROOT.ildStyle.SetPalette(ROOT.kBird)
ROOT.ildStyle.SetNumberContours(256)

In [5]:
ROOT.gInterpreter.Declare("#include \"WWTools.h\"")
# ROOT.gInterpreter.Declare("#include <podio/GenericParameters.h>")

True

In [6]:
%%cpp
using namespace ROOT::VecOps;

In [7]:
# tree = ROOT.TTree()
file = ROOT.TFile("data/paris/nano/all.root")
tree = file["events"]
# tree.AddFriend("events", "data/paris/nano/all.root")
tree.AddFriend("events", "data/paris/nano/all_params_flat.root")

df = ROOT.RDataFrame(tree)

n_bins = 100
x_angle = 0.014 # rad

# TODO implement
beamPol1 = 0.
beamPol2 = 0.
lumi = 1000. # inverse fb

In [8]:
df.Describe()

Dataframe from TTree events in file data/paris/nano/all.root
with friend
  events data/paris/nano/all_params_flat.root

Property                Value
--------                -----
Columns in total           93
Columns from defines        0
Event loops run             0
Processing slots            4

Column                                          Type                                                            Origin
------                                          ----                                                            ------
e_charge                                        Float_t                                                         Dataset
e_lvec                                          ROOT::Math::LorentzVector<ROOT::Math::PxPyPzE4D<double> >       Dataset
e_lvec.fCoordinates                             ROOT::Math::PxPyPzE4D<double>                                   Dataset
e_lvec.fCoordinates.fT                          Double_t                                             

In [9]:
# do weight calculations: for now only unpolarized
# should happen before any cuts
total_events = tree.GetEntries()

df = df.Define("pol_weight", "1./4.")
# df = df.Define("lumi_weight", "1")
df = df.Define("lumi_weight", f"{lumi} * params_crossSection / {total_events}.")
df = df.Define("weight", "pol_weight * lumi_weight")

In [10]:
# unboost crossing angle (as a variation?)
# reconstruct neutrino: missing E, w/o ovl, w/ overlay
# ISR? crossing angle, sqrt(s)
# build MC and reconstructed W, reconstructed W will vary

# df = df.Define("sqrt_s_E", "Parameters.get<float>(\"Energy\").value()")
df = df.Define("sqrt_s_E", "params_Energy")
# XXX: ignoring the electron mass here
df = df.Define("sqrt_s_lvec", f"ROOT::Math::PxPyPzEVector(sqrt_s_E * sin({x_angle}/2.), 0, 0, sqrt_s_E)")

# TODO correct electron mass back to m_e, maybe as a variation
# df = df.Redefine("e_lvec", "vec = PxPyPzEVector(e_lvec), vec.SetE")

# FIXME debug
df = df.Define("visible_lvec", "e_lvec + isr1_lvec + isr2_lvec + ovl_lvec + had_sum_lvec")
# TODO: add 3 variations: no removal, naive removal (Andre), cheated removal
# Andre's cut: log_10 Pt > 0.2 sin theta + something
df = df.Define("identified_ovl_lvec", "ROOT::Math::PxPyPzEVector()")
# TODO variations: none, fitted/reconstructed, cheated ISR goes here
df = df.Define("identified_isr_lvec", "ROOT::Math::PxPyPzEVector()")


In [11]:
df = df.Vary("identified_ovl_lvec", "RVec<ROOT::Math::PxPyPzEVector>({ovl_lvec})", ["cheated"], "ovl")
df = df.Vary("identified_isr_lvec", "RVec<ROOT::Math::PxPyPzEVector>({isr1_lvec + isr2_lvec})", ["cheated"], "isr")

In [12]:

df = df.Define("visible_corrected_lvec", "visible_lvec - identified_ovl_lvec - identified_isr_lvec")
df = df.Define("missing_E_lvec", "sqrt_s_lvec - visible_corrected_lvec")
df = df.Define("reco_nu_lvec", "missing_E_lvec")
# TODO: correct nu mass back to 0?

df = df.Define("delta_nu", "nu_mc_sum_lvec - reco_nu_lvec")

In [13]:
df = df.Define("W_lep_lvec", "e_lvec + reco_nu_lvec")
df = df.Define("W_had_lvec", "visible_corrected_lvec - e_lvec")

In [14]:
df = df.Define("W_lep_invM", "W_lep_lvec.mass()")
df = df.Define("W_had_invM", "W_had_lvec.mass()")

In [15]:
df = df.Define("nu_invM", "reco_nu_lvec.M()")
df = df.Define("mc_nu_invM", "nu_mc_sum_lvec.M()")

df = df.Define("delta_nu_E", "delta_nu.E()")
df = df.Define("delta_nu_P", "delta_nu.P()")
df = df.Define("delta_nu_Px", "delta_nu.Px()")
df = df.Define("delta_nu_Py", "delta_nu.Py()")
df = df.Define("delta_nu_Pz", "delta_nu.Pz()")

In [16]:
h_W_lep_invM = df.Histo1D(("", ";W_lep_invM [GeV]", 300, -100., 200.), "W_lep_invM", "weight")
h_W_had_invM = df.Histo1D(("", ";W_had_invM [GeV]", 200, 0., 200.), "W_had_invM", "weight")

v_W_lep_invM = ROOT.RDF.Experimental.VariationsFor(h_W_lep_invM)
v_W_had_invM = ROOT.RDF.Experimental.VariationsFor(h_W_had_invM)

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x55948e773690
OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x55948e7571d0


In [17]:
h_nu_invM = df.Histo1D("nu_invM")
h_mc_nu_invM = df.Histo1D("mc_nu_invM")

h_delta_nu_E = df.Histo1D("delta_nu_E")
h_delta_nu_P = df.Histo1D("delta_nu_P")
h_delta_nu_Px = df.Histo1D("delta_nu_Px")
h_delta_nu_Py = df.Histo1D("delta_nu_Py")
h_delta_nu_Pz = df.Histo1D("delta_nu_Pz")

In [18]:
c_W_lep_invM = ROOT.TCanvas()
h_W_lep_invM.Draw()
v_W_lep_invM["ovl:cheated"].SetMarkerColor(ROOT.kRed)
v_W_lep_invM["ovl:cheated"].Draw("same")
# v_W_lep_invM["isr:cheated"].SetMarkerColor(ROOT.kBlue)
# v_W_lep_invM["isr:cheated"].Draw("same")
c_W_lep_invM.Draw()

c_W_had_invM = ROOT.TCanvas()
h_W_had_invM.Draw()
v_W_had_invM["ovl:cheated"].SetMarkerColor(ROOT.kRed)
v_W_had_invM["ovl:cheated"].Draw("same")
c_W_had_invM.Draw()

In [19]:
c_nu_invM = ROOT.TCanvas()
h_nu_invM.Draw()
c_nu_invM.Draw()

c_mc_nu_invM = ROOT.TCanvas()
h_mc_nu_invM.Draw()
c_mc_nu_invM.Draw()

c_delta_nu_E = ROOT.TCanvas()
h_delta_nu_E.Draw()
c_delta_nu_E.Draw()

c_delta_nu_P = ROOT.TCanvas()
h_delta_nu_P.Draw()
c_delta_nu_P.Draw()

c_delta_nu_Px = ROOT.TCanvas()
h_delta_nu_Px.Draw()
c_delta_nu_Px.Draw()

c_delta_nu_Py = ROOT.TCanvas()
h_delta_nu_Py.Draw()
c_delta_nu_Py.Draw()

c_delta_nu_Pz = ROOT.TCanvas()
h_delta_nu_Pz.Draw()
c_delta_nu_Pz.Draw()